# YOLOv8 Training and Conversion to RKNN

In [ ]:
root_path = '/content'
%cd {root_path}
import os
root_path = os.getcwd()

!git clone https://github.com/airockchip/ultralytics_yolov8 ultralytics
%cd ultralytics

!pip install -e .

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

## Downloading the dataset
Input your Roboflow API key below. It can be obtained [here](https://app.roboflow.com/settings/api).
You can use your own dataset, the rest of the notebook should work with any number of classes, as long as the project is of "object detection" type.

In [ ]:
!mkdir {root_path}/datasets
%cd {root_path}/datasets

!pip install roboflow -q

from roboflow import Roboflow

rf = Roboflow(api_key="roboflowKey")
project = rf.workspace("YOUR_WORKSPACE_HERE").project("YOUR_PROJECT_HERE")
dataset = project.version(5).download("yolov8")


In [ ]:
import os
import re
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
        print("Wrote successfully to " + line)

@register_line_cell_magic
def replaceAllInFile(line, cell):
    filename = line.strip()
    replacements = eval(cell)  # Assuming input is a valid Python expression
    with open(filename, 'r') as f:
        file_content = f.read()
    for replaced, with_this in replacements:
        file_content = re.sub(replaced, with_this, file_content)
    with open(filename, 'w') as f:
        f.write(file_content)
    print(f"Replaced successfully in {filename}")

In [ ]:
%cat {dataset.location}/data.yaml

#### This is needed to fix the location of the various training directories, as it might not be accurate.  You can rerun the code block above to double check.

In [ ]:
%%replaceAllInFile {dataset.location}/data.yaml

[
    ('test: ..', 'test: ' + dataset.location),
    ('train: ..', 'train: ' + dataset.location),
    ('val: ..', 'val: ' + dataset.location),
]


In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
print(f"num_classes: {num_classes}")
%cd {root_path}/ultralytics

#### This is the custom template that we need to train on, it lets us eventually convert to rknn.

In [ ]:
%%writetemplate custom_yolov8.yaml
# Ultralytics YOLO 🚀, AGPL-3.0 license
# YOLOv8 object detection model with P3-P5 outputs. For Usage examples see https://docs.ultralytics.com/tasks/detect

# Parameters
nc: {num_classes}  # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024]  # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  s: [0.33, 0.50, 1024]  # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
  m: [0.67, 0.75, 768]   # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
  l: [1.00, 1.00, 512]   # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  x: [1.00, 1.25, 512]   # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs
activation: nn.ReLU()
# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]  # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]  # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]  # cat backbone P4
  - [-1, 3, C2f, [512]]  # 12

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]  # cat backbone P3
  - [-1, 3, C2f, [256]]  # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]]  # cat head P4
  - [-1, 3, C2f, [512]]  # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]]  # cat head P5
  - [-1, 3, C2f, [1024]]  # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]]  # Detect(P3, P4, P5)

## Training
You can adjust the following settings:
- model: one of [yolov8n, yolov8s, yolov8m, yolov8l, yolov8x]
- epochs: How many iterations to train for
- image_size: The input size of the images fed to the model. Should be a multiple of 32.
- batch: Number of samples per epoch. You should set this to the highest number possible without the training taking too much memory (it would crash if that happens, which is ok, just lower the number and try again)
- cache: Enables caching of dataset images in memory (True/ram), on disk (disk), or disables it (False)
- patience: After how many epochs without improvement to stop the training

In [ ]:
%cd {root_path}/ultralytics
model = "yolov8s"
epochs = 100
image_size = 640
batch=48
cache=True
patience=50
!yolo epochs={epochs} task=detect mode=train model=./custom_{model}.yaml data={dataset.location}/data.yaml imgsz={image_size} cache={cache} batch={batch} patience={patience}

In [ ]:
latest_modified_time = 0
latest = None

for foldername, subfolders, filenames in os.walk(root_path):
    for filename in filenames:
        if filename == "best.pt":
            file_path = os.path.join(foldername, filename)
            modified_time = os.path.getmtime(file_path)
            if modified_time > latest_modified_time:
                latest_modified_time = modified_time
                latest = file_path
print(latest)

### Installing RKNN Toolkit 2

In [ ]:
!wget https://github.com/rockchip-linux/rknn-toolkit2/raw/2c2d03def0c0908c86985b8190e973976ecec74c/rknn-toolkit2/packages/rknn_toolkit2-1.6.0+81f21f4d-cp310-cp310-linux_x86_64.whl
!pip install ./rknn_toolkit2-1.6.0+81f21f4d-cp310-cp310-linux_x86_64.whl

In [ ]:
%cd {root_path}
!git clone https://github.com/airockchip/rknn_model_zoo/
%cd rknn_model_zoo
!git checkout eaa94d6f57ca553d493bf3bd7399a070452d2774
%cd examples/yolov8/python

## Sample Images For Quantization

In [ ]:
%%writefile imgs.txt
imgs/1.jpg
imgs/2.jpg
imgs/3.jpg
imgs/4.jpg
imgs/5.jpg
imgs/6.jpg
imgs/7.jpg
imgs/8.jpg
imgs/9.jpg
imgs/10.jpg
imgs/11.jpg
imgs/12.jpg
imgs/13.jpg
imgs/14.jpg
imgs/15.jpg
imgs/16.jpg
imgs/17.jpg
imgs/18.jpg
imgs/19.jpg
imgs/20.jpg

In [ ]:
import os
import shutil
import random
import glob

def copy_and_rename_images(source_folder, destination_folder, n):
    if not os.path.exists(source_folder):
        print(f"Source folder '{source_folder}' does not exist.")
        return
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    image_files = glob.glob(os.path.join(source_folder, '*.jpg'))
    selected_images = random.sample(image_files, min(n, len(image_files)))
    for i, image_path in enumerate(selected_images, start=1):
        destination_path = os.path.join(destination_folder, f'{i}.jpg')
        shutil.copy(image_path, destination_path)
    print(f"{min(n, len(image_files))} random images copied from '{source_folder}' to '{destination_folder}' and renamed.")
copy_and_rename_images(dataset.location+"/test/images" , "imgs", 20)

In [ ]:
%%replaceAllInFile {root_path}/rknn_model_zoo/examples/yolov8/python/convert.py
[
    ('../../../datasets/COCO/coco_subset_20.txt', 'imgs.txt'),
]

## Exporting to ONNX
This is an intermediate step between the PyTorch model and the RKNN model.

In [ ]:
%cd {root_path}/ultralytics
!yolo mode=export format=rknn model={latest}

In [ ]:
ex_path = '.'.join(latest.split('.')[:-1]) + '.onnx'
print(ex_path)

## Quantization
Here you choose whether to perform quantization, which makes the model lighter and faster, by converting all 32/16 bit floates in the model into 8 bit ints, which costs performance.

In [ ]:
to_quantize = True # @param {type: "boolean"}

## Exporting to RKNN - Final step🎉

In [ ]:
%cd {root_path}/rknn_model_zoo/examples/yolov8/python
quant_code = "i8" if to_quantize else "fp"
output_model = f"{root_path}/{dataset.name}-{model}-{image_size}-{quant_code}.rknn"
!python convert.py {ex_path} rk3588 {quant_code} {output_model}

This notebook was forked from [laviRZ's](https://github.com/lavirz/photonvision/blob/master/devTools/yolov8-to-rknn.ipynb)